<a href="https://colab.research.google.com/github/benza613/Sentiment-Analysis-using-Deep-Learning/blob/master/notebooks/LexicalAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/gdrive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

In [0]:
%cd Sentiment-Analysis-using-Deep-Learning/

In [0]:
!git pull https://7519148727b530a7734e756bada859a5811db262@github.com/benza613/Sentiment-Analysis-using-Deep-Learning.git

In [0]:
!git remote set-url origin https://7519148727b530a7734e756bada859a5811db262@github.com/benza613/Sentiment-Analysis-using-Deep-Learning.git

In [0]:
!git push origin master

In [0]:
# Clone github repository setup
# import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'My Drive/SNLP' 
# replace with your Github username 
GIT_USERNAME = "benza613" 
# definitely replace with your
GIT_TOKEN = "7519148727b530a7734e756bada859a5811db262"  
# Replace with your github repository in this case we want 
# to clone deep-learning-v2-pytorch repository
GIT_REPOSITORY = "Sentiment-Analysis-using-Deep-Learning" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path 
!mkdir "{PROJECT_PATH}"    

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
print("GIT_PATH: ", GIT_PATH)

In [0]:
  !git config --global user.email "rjayar4@uic.edu"
  !git config --global user.name "rakshitharj"

In [0]:
#unzip the train and test data
from zipfile import ZipFile
# Create a ZipFile Object and load abc.zip in it
#with ZipFile('Train_20000_Apr-03-2020_23-22.zip', 'r') as zipObj:
with ZipFile('Test_20000_Apr-03-2020_23-22.zip', 'r') as zipObj:
# Extract all the contents of zip file in same directory
  zipObj.extractall()

In [0]:
import sys
import pickle
import random

#the preprocessed files are stored in Data_Balanced_20000_Apr-03-2020_23-22 folder 
#https://github.com/abdulfatir/twitter-sentiment-analysis/blob/master/code/preprocess.py
TRAIN_PROCESSED_FILE = 'Data_Balanced_20000_Apr-03-2020_23-22/Train_20000_Apr-03-2020_23-22.csv'
TEST_PROCESSED_FILE = 'Data_Balanced_20000_Apr-03-2020_23-22/Test_20000_Apr-03-2020_23-22.csv'

#the text files with positive and negative words in data_pos_neg folder
POSITIVE_WORDS_FILE = 'data_pos_neg/positive_words.txt'
NEGATIVE_WORDS_FILE = 'data_pos_neg/negative_words.txt'

In [0]:
TRAIN = True

def save_results_to_csv(results, csv_file):
    ''' Save list of type [(reviewerID, prediction)] to csv in Kaggle format '''
    with open(csv_file, 'w') as csv:
        csv.write('id,prediction\n')
        for reviewerID, pred in results:
            csv.write(reviewerID)
            csv.write(',')
            csv.write(str(pred))
            csv.write('\n')

def file_to_wordset(filename):
    ''' Converts a file with a word per line to a Python set '''
    words = []
    with open(filename, 'r') as f:
        for line in f:
            words.append(line.strip())
    return set(words)

def classify(processed_csv, test_file=True, **params):
    positive_words = file_to_wordset(params.pop('positive_words'))
    print("positive")
    negative_words = file_to_wordset(params.pop('negative_words'))
    predictions = []
    with open(processed_csv, 'r') as csv:
        for line in csv:
           # print("line",line)
            if test_file:
                overall, reviewTime1, reviewTime2, reviewerID,asin,reviewText,*rest = line.strip().split(',')
            else:
               # print(line.strip().split(','))
                overall, reviewTime1, reviewTime2, reviewerID, asin, reviewText,*rest = line.strip().split(',')
            pos_count, neg_count = 0, 0
            for word in reviewText.split():
                #print("word",word)
                if word in positive_words:
                    pos_count += 1
                elif word in negative_words:
                    neg_count += 1
            # print pos_count, neg_count
            prediction = 1 if pos_count >= neg_count else 0
            if test_file:
                predictions.append((reviewerID, prediction))
            else:
                predictions.append((reviewerID, prediction))
            #print("predictions",predictions)
    return predictions


if __name__ == '__main__':
    if TRAIN:
        predictions = classify(TRAIN_PROCESSED_FILE, test_file=(not TRAIN), positive_words=POSITIVE_WORDS_FILE, negative_words=NEGATIVE_WORDS_FILE)
        #print(predictions)
        save_results_to_csv(predictions, 'train_baseline.csv')
        #correct = sum([1 for p in predictions if p[1] == 0]) * 100.0 / len(predictions)
        #print('Correct = %.2f%%' % correct)
    else:
        predictions = classify(TEST_PROCESSED_FILE, test_file=(not TRAIN), positive_words=POSITIVE_WORDS_FILE, negative_words=NEGATIVE_WORDS_FILE)
        save_results_to_csv(predictions, 'test_baseline.csv')